# Multiclass SVM 구현

In [25]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.svm import SVC
from sklearn.multiclass import OneVsOneClassifier, OneVsRestClassifier
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score

#IRIS 데이터 로드
iris =  sns.load_dataset('iris') 
X= iris.iloc[:,:4] #학습할데이터
y = iris.iloc[:,-1] #타겟
print(y)

0         setosa
1         setosa
2         setosa
3         setosa
4         setosa
         ...    
145    virginica
146    virginica
147    virginica
148    virginica
149    virginica
Name: species, Length: 150, dtype: object


In [26]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=48)

In [27]:
def standardization(train, test):
    scaler = StandardScaler()
    train = scaler.fit_transform(train)
    test = scaler.transform(test)
    return train, test

X_train, X_test = standardization(X_train, X_test)

In [28]:
X_train

array([[ 0.78522493,  0.32015325,  0.77221097,  1.04726529],
       [-0.26563371, -1.29989934,  0.0982814 , -0.11996537],
       [ 0.43493872,  0.78302542,  0.94069336,  1.43634218],
       [-0.84944407,  0.78302542, -1.24957775, -1.28719604],
       [-0.38239578, -1.7627715 ,  0.15444219,  0.13941922],
       [ 0.55170079, -0.374155  ,  1.05301496,  0.7878807 ],
       [ 0.31817664, -0.14271892,  0.65988937,  0.7878807 ],
       [ 0.20141457, -0.374155  ,  0.43524618,  0.39880381],
       [-1.66677857, -0.14271892, -1.36189934, -1.28719604],
       [-0.14887164, -0.60559109,  0.21060299,  0.13941922],
       [-0.14887164, -1.06846325, -0.12636179, -0.24965767],
       [ 0.31817664, -0.60559109,  0.15444219,  0.13941922],
       [ 0.66846286, -0.83702717,  0.88453256,  0.91757299],
       [ 0.0846525 , -0.14271892,  0.77221097,  0.7878807 ],
       [-0.49915786, -0.14271892,  0.43524618,  0.39880381],
       [-0.26563371, -0.60559109,  0.65988937,  1.04726529],
       [ 2.18636979,  1.

In [29]:
X_test

array([[-0.14887164, -0.374155  ,  0.26676379,  0.13941922],
       [ 0.31817664, -0.60559109,  0.54756778,  0.00972692],
       [ 0.31817664, -1.06846325,  1.05301496,  0.26911151],
       [-1.5500165 , -1.7627715 , -1.36189934, -1.15750374],
       [ 0.0846525 ,  0.32015325,  0.60372857,  0.7878807 ],
       [ 0.78522493, -0.14271892,  0.99685416,  0.7878807 ],
       [-0.84944407,  1.70876975, -1.24957775, -1.15750374],
       [ 0.20141457, -0.14271892,  0.60372857,  0.7878807 ],
       [-0.38239578,  2.63451409, -1.30573855, -1.28719604],
       [-0.38239578, -1.29989934,  0.15444219,  0.13941922],
       [ 0.66846286,  0.08871717,  0.99685416,  0.7878807 ],
       [-0.38239578,  1.0144615 , -1.36189934, -1.28719604],
       [-0.49915786,  0.78302542, -1.13725615, -1.28719604],
       [ 0.43493872, -0.60559109,  0.60372857,  0.7878807 ],
       [ 0.55170079, -1.7627715 ,  0.37908538,  0.13941922],
       [ 0.55170079,  0.55158933,  0.54756778,  0.52849611],
       [-1.19973028,  0.

## OneVsRest - decision function

In [33]:
import numpy as np
from sklearn.svm import SVC

class OnevsRestSVM:
    def __init__(self):
        self.classes = None  # 클래스 개수 
        self.clfs = {}  # 분류기 저장
        self.indexing = {} # 클래스 레이블이 str 형태이므로 0,1,2와 같이 정수로 인덱싱

    def fit(self, X_train, y_train, kernel='rbf', C=1, gamma='scale'):
        self.classes = np.unique(y_train)
        for index, class_ in enumerate(self.classes):
            self.indexing[class_] = index  # ✅ 
            y_binary = np.where(y_train == class_, 1, -1)
            ovr_clf = SVC(kernel=kernel, C=C, gamma=gamma, probability=True)
            ovr_clf.fit(X_train, y_binary)
            self.clfs[class_] = ovr_clf

    def predict(self, X_test):
        pred = np.zeros((X_test.shape[0], len(self.classes)))
        for class_, clf in self.clfs.items():
            index = self.indexing[class_]  
            pred[:, index] = clf.decision_function(X_test)
        return self.classes[np.argmax(pred, axis=1)]

In [34]:
svm = OnevsRestSVM()
svm.fit(X_train, y_train)
ovr_pred=svm.predict(X_test)

In [35]:
accuracy_score(y_test, ovr_pred)

0.9666666666666667

### 라이브러리 결과와 비교

In [36]:
# OVR 전략 
ovr_clf = OneVsRestClassifier(SVC())
ovr_clf.fit(X_train, y_train)
y_pred = ovr_clf.predict(X_test)

accuracy_score(y_test, y_pred)

0.9666666666666667